In [1]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np 
import time, math, random
from typing import Tuple

import gymnasium as gym

In [2]:
env = gym.make('MountainCar-v0', render_mode = "human")

In [3]:
n_bins = ( 3 , 4 )

# x position and velocity
lower_bounds = [ env.observation_space.low[0], env.observation_space.low[1] ]
upper_bounds = [ env.observation_space.high[0], env.observation_space.high[1] ]

def discretizer( x_pos, velocity ) -> Tuple[int,...]:
    """Convert continues state intro a discrete state"""
    est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bounds ])
    return tuple(map(int,est.transform([[x_pos, velocity]])[0]))

In [4]:
Q_table = np.zeros(n_bins + (env.action_space.n,))
Q_table.shape

(3, 4, 3)

In [5]:
def policy( state : tuple ):
    """Choosing action based on epsilon-greedy policy"""
    return np.argmax(Q_table[state])

In [6]:
def new_Q_value( reward : float ,  new_state : tuple , discount_factor=1 ) -> float:
    """Temperal diffrence for updating Q-value of state-action pair"""
    future_optimal_value = np.max(Q_table[new_state])
    learned_value = reward + discount_factor * future_optimal_value
    return learned_value

In [7]:
# Adaptive learning of Learning Rate
def learning_rate(n : int , min_rate=0.01 ) -> float  :
    """Decaying learning rate"""
    return max(min_rate, min(1.0, 1.0 - math.log10((n + 1) / 25)))

In [8]:
def exploration_rate(n : int, min_rate= 0.1 ) -> float :
    """Decaying exploration rate"""
    return max(min_rate, min(1, 1.0 - math.log10((n  + 1) / 25)))

In [9]:
n_episodes = 3 
for e in range(n_episodes):
    print(e)
    
    reset_vals = env.reset()[0]
    # Discretize state into buckets
    current_state, done = discretizer(reset_vals[0], reset_vals[1]), False
    
    while done==False:
        
        # policy action 
        action = policy(current_state) # exploit
        
        # insert random action
        if np.random.random() < exploration_rate(e) : 
            action = env.action_space.sample() # explore 
         
        # increment enviroment
        obs, reward, done, _ = env.step(action)[0:4]
        new_state = discretizer(*obs)
        
        # Update Q-Table
        lr = learning_rate(e)
        learnt_value = new_Q_value(reward , new_state )
        old_value = Q_table[current_state][action]
        Q_table[current_state][action] = (1-lr)*old_value + lr*learnt_value
        
        current_state = new_state
        
        # Render the cartpole environment
        env.render()

0


KeyboardInterrupt: 

In [ ]:
?render